In [1]:
import pandas as pd

import numpy as np

from pandas import DataFrame,Series

import matplotlib.pyplot as plt

import seaborn as sns

import lightgbm as lgb

import gc

import time

from sklearn import preprocessing

from sklearn.metrics import precision_recall_curve

from sklearn.model_selection import train_test_split

from sklearn.feature_selection import VarianceThreshold

In [2]:
train_x=pd.read_csv('data/atec_anti_fraud_train.csv')
test = pd.read_csv('data/atec_anti_fraud_test_b.csv')

In [3]:
train_x.sort_values(by='date',inplace=True)

train_x.drop("id",axis=1,inplace=True)
train_x.loc[train_x['label']==-1,'label']=1

train_y=train_x["label"]
train_x.drop("label",axis=1,inplace=True)

test_id=test["id"].copy()
test.drop("id",axis=1,inplace=True)

train_size = len(train_x)

In [4]:
all_data = train_x.append(test) 
del train_x,test
gc.collect()

28

In [5]:
#筛选之后剩下的需要使用的特征
col_need=['f1', 'f2', 'f3', 'f4', 'f6', 'f7', 'f8', 'f9', 'f11',
       'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f161',
       'f162', 'f163', 'f164', 'f165', 'f211', 'f212', 'f213', 'f214',
       'f215', 'f216', 'f217', 'f218', 'f219', 'f220', 'f221', 'f222',
       'f223', 'f224', 'f225', 'f226', 'f227', 'f228', 'f229', 'f230',
       'f231', 'f232', 'f233', 'f234', 'f235', 'f236', 'f237', 'f238',
       'f239', 'f240', 'f241', 'f242', 'f243', 'f244', 'f245', 'f246',
       'f247', 'f248', 'f249', 'f250', 'f251', 'f252', 'f253']

#这个函数用于将其余的不需要的特征列删除
def preprocess_likeo(df,col_need):
    num =0
    drop_cols=[]
    for col in df.columns:
        if(col not in col_need):    
            drop_cols.append(col)
            num= num +1
    df.drop(drop_cols,axis=1,inplace=True)
    print('drop ',num, " features in data set")
    return df


In [6]:
all_data = preprocess_likeo(all_data,col_need)

drop  233  features in data set


In [7]:
gc.collect()

7

In [8]:
#分别使用均值填充，插值填充和判定是否空值的方式填充连续型特征
def mean_fillna(df,features):
    for feature in features:
        df[feature+'_mean']=df[feature].fillna(df[feature].mean())
    return df

def interpolate_fillna(df,features):
    for feature in features:
        df[feature]=df[feature].interpolate()
    return df
        
def isnan_fillna(df,features):
    for feature in features:
        df[feature+'_isnan']=df[feature]
        df.loc[(df[feature].isnull()),feature+'_isnan']=1
        df.loc[(df[feature].notnull()),feature+'_isnan']=0
    return df
def median_fillna(df,features):
    for feature in features:
        df[feature+'_meadian']=df[feature].median()
    return df

#连续特征的处理方式
def continues_fillna(df,features):
    df=median_fillna(df,features)
    #df=interpolate_fillna(df,features)
    return df


#使用-1填充离散型特征，并生成是否为空的特征列
def category_fillna(df,category_cols):
    for category_col in category_cols:
        df[category_col].fillna(-1,inplace=True)
    return df

#为train和test数据标记是否是离散值，当数据unquie唯一值个数超过threld的时候，则被认为是连续特征，否则是离散特征。
#并且填充缺失值
def set_cate_type_and_fillna(train_x,test,threld=80):
    #给离散列转换类型,得到离散值
    data_clo_unique_num=pd.DataFrame(train_x.apply(lambda x:len(x.unique())),columns=['unique'])
    for i in data_clo_unique_num.index:
        print(i," ",data_clo_unique_num.loc[i,'unique'])
    cate_cols=set(data_clo_unique_num[data_clo_unique_num['unique']<threld].index)
    cont_cols=set(train_x.columns)-cate_cols
    for i in cate_cols:
        train_x=train_x.astype('category')
        test=test.astype('category') 
    train_x=category_fillna(train_x,cate_cols)
    test=category_fillna(test,cate_cols)
    train_x=continues_fillna(train_x,cont_cols)
    test=continues_fillna(test,con_cols)

In [10]:
train_x = all_data[:train_size]
test = all_data[train_size:]
del all_data
gc.collect()

0

In [11]:
print("the size of train set is "+str(train_x.shape[0])+", the size of test set is "+str(test.shape[0]))
set_cate_type_and_fillna(train_x,test,80)

the size of train set is 994731, the size of test set is 500538
f1   3
f2   3
f3   3
f4   3
f6   5
f7   8
f8   3
f9   3
f11   3
f12   3
f13   3
f14   3
f15   3
f16   3
f17   3
f18   3
f19   3
f161   195
f162   255
f163   251
f164   250
f165   195
f211   196
f212   256
f213   238
f214   251
f215   292
f216   303
f217   303
f218   303
f219   32
f220   33
f221   33
f222   33
f223   135
f224   159
f225   157
f226   161
f227   252
f228   249
f229   255
f230   187
f231   258
f232   256
f233   256
f234   273
f235   303
f236   303
f237   396
f238   667
f239   78
f240   276
f241   274
f242   278
f243   281
f244   249
f245   303
f246   295
f247   303
f248   342
f249   160
f250   278
f251   253
f252   268
f253   275


ValueError: fill value must be in categories

In [12]:
#评价函数，可用于普通的sklearn包算法的结果分析，如GBDT
def feval_spec(preds, train_data):
    from sklearn.metrics import roc_curve
    fpr, tpr, threshold = roc_curve(train_data, preds)
    tpr0001 = tpr[fpr <= 0.001].max()
    tpr001 = tpr[fpr <= 0.005].max()
    tpr005 = tpr[fpr <= 0.01].max()
    #tpr01 = tpr[fpr.values <= 0.01].max()
    tprcal = 0.4 * tpr0001 + 0.3 * tpr001 + 0.3 * tpr005
    return tprcal

In [13]:
#用于lightgbm的评价函数
def feval_spec_train(preds, train_data):
    from sklearn.metrics import roc_curve
    fpr, tpr, threshold = roc_curve(train_data.get_label(), preds)
    tpr0001 = tpr[fpr <= 0.001].max()
    tpr001 = tpr[fpr <= 0.005].max()
    tpr005 = tpr[fpr <= 0.01].max()
    #tpr01 = tpr[fpr.values <= 0.01].max()
    tprcal = 0.4 * tpr0001 + 0.3 * tpr001 + 0.3 * tpr005
    return 'spec_cal',tprcal,True

In [22]:
#Kfolds将数据根据索引分为k份，可以用于生成第一层输出的时候，保证各折数据之间没有干扰
# GroupSelect将Kfolds中的第i折数据索引作为预测集pre_set，其余的作为训练集train_set。
# TrainSet针对训练数据的i折，得到训练数据和第二层的输入数据
def Kfolds(x_index,k=5,seed=1):
    np.random.seed(seed)
    xL=np.array_split(np.random.choice(x_index,len(x_index),replace=False),k)
    return xL

def GroupSelect(xL,i=0):
    xLc=xL.copy()
    pre_index=list(xLc.pop(i))
    train_index=sum([list(x) for x in xLc],[])
    return train_index,pre_index

def TrainSet(x,y,xL,i=0):
    train_index,pre_index=GroupSelect(xL,i)
    train_x,pre_x=x.loc[train_index],x.loc[pre_index]
    train_y,pre_y=y.loc[train_index],y.loc[pre_index]
    return train_x,train_y,pre_x,pre_y

#K折交叉验证，可用于sklearn传统机器学习算法
def KflodsTrain(x,y,test,k=5,classifier=lgb.LGBMClassifier()):
    xL=Kfolds(np.array(x.index),k)
    #predict_proba=pd.DataFrame()
    test_predict_proba=pd.DataFrame()
    for i in range(k):
        print("begin the "+str(i)+" of "+str(k)+" kflods training...")
        train_x,train_y,pre_x,pre_y=TrainSet(x,y,xL,i)
        model=classifier.fit(train_x,train_y)
        predict_res=pd.Series(model.predict_proba(pre_x)[:, 1],index=pre_x.index)
        train_metric=feval_spec(model.predict_proba(train_x)[:,1],train_y)
        valid_metri=feval_spec(predict_res,pre_y)
        print("The ",i," times res: train set spe_val:",train_metric,", validation set sep_val: ",valid_metri)
        #predict_proba=pd.concat([predict_proba,predict_res])
        test_predict_proba=pd.Series(model.predict_proba(test)[:,1],index=test.index)
    #print("The output data of classifier: ",type(classifier), " is ready for stacking...")
    #print("the size  of data is ",predict_proba.shape[0])
    test_predict_proba_mean=test_predict_proba.mean(axis=1)
    return test_predict_proba_mean

#K折交叉验证算法，用于lightgbm算法
def KflodsTrain_light(x,y,test,k=5,lgb_params=None):
    if lgb_params==None:
        lgb_params = {
            'boosting_type': 'gbdt', 
            'objective': 'binary', 
            'nthread': -1, 
            'metric': 'auc',
            'num_leaves': 7,  # -1 means no limit
            'min_child_samples': 1000,  # Minimum number of data need in a child(min_data_in_leaf)  
        }
    xL=Kfolds(np.array(x.index),k)
    #predict_proba=pd.DataFrame()
    test_predict_proba=pd.DataFrame()
    for i in range(k):
        print("begin the "+str(i)+" of "+str(k)+" kflods training...")
        train_x,train_y,pre_x,pre_y=TrainSet(x,y,xL,i)
        xgtrain = lgb.Dataset(train_x,train_y)
        xgvalid = lgb.Dataset(pre_x,pre_y)
        evals_results={}
        bst1 = lgb.train(lgb_params,xgtrain,valid_sets=[xgtrain,xgvalid],valid_names=['train','valid'],evals_result=evals_results,
                    num_boost_round=200,early_stopping_rounds=30, verbose_eval=False,feval=feval_spec_train)        
        print('The ',i,' times running....')
        print('Best ite and score ',bst1.best_iteration,bst1.best_score)
        #predict_proba=pd.concat([predict_proba,predict_res])
        test_predict_proba=pd.Series(bst1.predict(test,num_iteration=bst1.best_iteration),index=test.index)
        
    #print("The output data of classifier: ",type(classifier), " is ready for stacking...")
    #print("the size  of data is ",predict_proba.shape[0])
    print(test_predict_proba)
    return test_predict_proba
#     test_predict_proba_mean=test_predict_proba.mean(axis=1)
#     return test_predict_proba_mean



In [23]:
#设置lightgbm参数之后，调用上面的方法即可
total_res=pd.DataFrame()
total_res['id']=test_id
lgb_params = {
            'boosting_type': 'gbdt', 
            'objective': 'binary', 
            'nthread': -1, 
            'metric': 'auc',
            'bagging_fraction':0.8,
            'feature_fraction':0.8,
            'num_leaves': 250,  # -1 means no limit
            'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)  
        }


In [25]:
total_res["score"]=KflodsTrain_light(train_x,train_y,test,10,lgb_params)

begin the 0 of 10 kflods training...
The  0  times running....
Best ite and score  42 defaultdict(<class 'dict'>, {'valid': {'auc': 0.9517678597534437, 'spec_cal': 0.3059649122807018}, 'train': {'auc': 0.9730200833072992, 'spec_cal': 0.42521635727026497}})
begin the 1 of 10 kflods training...
The  1  times running....
Best ite and score  39 defaultdict(<class 'dict'>, {'valid': {'auc': 0.9514489389023241, 'spec_cal': 0.2954545454545454}, 'train': {'auc': 0.9710313006638056, 'spec_cal': 0.42190325348115887}})
begin the 2 of 10 kflods training...
The  2  times running....
Best ite and score  62 defaultdict(<class 'dict'>, {'valid': {'auc': 0.9465062185011118, 'spec_cal': 0.2848414985590778}, 'train': {'auc': 0.9834448957546803, 'spec_cal': 0.46703943885653787}})
begin the 3 of 10 kflods training...
The  3  times running....
Best ite and score  44 defaultdict(<class 'dict'>, {'valid': {'auc': 0.9505816389046823, 'spec_cal': 0.289512893982808}, 'train': {'auc': 0.9738251245096424, 'spec_ca

In [26]:
total_res.to_csv("submission_1.csv",index=False)